# Import

In [1]:
from run import *
from tint.metrics import mse, mae
import tint, captum
from tqdm import tqdm
import pandas as pd
from utils.explainer import *

In [2]:
from captum.attr import (
    DeepLift,
    GradientShap,
    IntegratedGradients,
    Lime
)

from tint.attr import (
    AugmentedOcclusion,
    DynaMask,
    Occlusion, 
    Fit, FeatureAblation
)

from tint.attr import (
    TemporalAugmentedOcclusion,
    TemporalIntegratedGradients,
    TemporalOcclusion,
    TimeForwardTunnel,
)

# Arguments

In [32]:
parser = get_parser()
argv = """
  --root_path ./dataset/illness/ \
  --data_path national_illness.csv \
  --model DLinear \
  --data custom \
  --use_gpu
  --features MS \
  --seq_len 36 \
  --label_len 12 \
  --pred_len 24 \
  --n_features 7
""".split()
args = parser.parse_args(argv)

# Disable cudnn if using cuda accelerator.
# Please see https://captum.ai/docs/faq#how-can-i-resolve-cudnn-rnn-backward-error-for-rnn-or-lstm-network
# args.use_gpu = False
initial_setup(args)

In [4]:
args.explainers = [
    "deep_lift",
    "gradient_shap",
    "integrated_gradients",
    "lime",
    "occlusion",
    "augmented_occlusion",
]

args.areas = [
    0.05, 0.1
]

In [5]:
explainer_name_map = {
    "deep_lift":DeepLift,
    "gradient_shap":GradientShap,
    "integrated_gradients":IntegratedGradients,
    "lime":Lime,
    "occlusion":Occlusion,
    "augmented_occlusion":AugmentedOcclusion,
    "dyna_mask":DynaMask,
    "feature_ablation":FeatureAblation
}

# Initialize

In [33]:
assert args.task_name == 'long_term_forecast', "Only long_term_forecast is supported for now"

setting = stringify_setting(args)

exp = Exp_Long_Term_Forecast(args)  # set experiments
_, dataloader = exp._get_data('test')
exp.load_best_model()

Use GPU: cuda:0
Experiments will be saved in ./results\national_illness_DLinear
test 73
Loading model from ./results\national_illness_DLinear\checkpoint.pth


In [34]:
model = exp.model
model.eval()
model.zero_grad()

# only need to output targets, sinec interpretation is based on outputs
assert not exp.args.output_attention

In [17]:
expl_metrics = ['mae', 'mse']
# expl_metrics = [getattr(tint.metrics, metric_name) for metric_name in expl_metrics]
areas = [0.05, 0.075, 0.1]

explainers = ['deep_lift', 'feature_ablation'] # explainers = args.explainers
explainers_map = dict()
for name in explainers:
    if name == 'augmented_occlusion':
        inputs = get_total_data(dataloader)
        explainers_map[name] = explainer_name_map[name](model, inputs)
    else:    
        explainers_map[name] = explainer_name_map[name](model)

# Evaluate

In [20]:
results = []
baseline_mode = "random" # "zeros", "aug"
result_columns = ['batch_index', 'explainer', 'metric', 'area', 'comp', 'suff']

progress_bar = tqdm(
    enumerate(dataloader), total=len(dataloader), disable=False
)
for batch_index, (batch_x, batch_y, batch_x_mark, batch_y_mark) in progress_bar:
    batch_x = batch_x.float().to(exp.device)
    batch_y = batch_y.float().to(exp.device)

    batch_x_mark = batch_x_mark.float().to(exp.device)
    batch_y_mark = batch_y_mark.float().to(exp.device)
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float()
    # outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    
    inputs = (batch_x, batch_x_mark)
    # baseline must be a scaler or tuple of tensors with same dimension as input
    baselines = get_baseline(inputs, baseline_mode)
    additional_forward_args = (dec_inp, batch_y_mark)

    # get attributions
    for name in explainers:
        explainer = explainers_map[name]
        attr = compute_attr(
            inputs, baselines, explainer, additional_forward_args, args
        )
    
        # get scores
        for area in areas:
            for metric_name in ['mae', 'mse']:
                metric = getattr(tint.metrics, metric_name)
                error_comp = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=True
                )
                
                error_suff = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=False
                )
           
                result_row = [batch_index, name, metric_name, area, error_comp, error_suff]
                # print(result_row)
                results.append(result_row)

100%|██████████| 3/3 [00:17<00:00,  5.79s/it]


In [21]:
results_df = pd.DataFrame(results, columns=result_columns)
results_df = results_df.groupby(['explainer', 'metric', 'area'])[['comp', 'suff']].aggregate('mean').reset_index()
# results_df.round(6).to_csv(os.path.join(result_folder, 'interpretation_results.csv'), index=False)
print(results_df)

           explainer metric   area        comp       suff
0          deep_lift    mae  0.050   35.556379  34.244208
1          deep_lift    mae  0.075   49.028145  21.650221
2          deep_lift    mae  0.100   61.907148  11.877960
3          deep_lift    mse  0.050   54.197291  54.057170
4          deep_lift    mse  0.075  102.314908  23.969013
5          deep_lift    mse  0.100  162.540655   8.745565
6   feature_ablation    mae  0.050   35.368984  34.748280
7   feature_ablation    mae  0.075   48.341543  21.994582
8   feature_ablation    mae  0.100   61.186089  12.291678
9   feature_ablation    mse  0.050   53.678370  55.767656
10  feature_ablation    mse  0.075   99.778231  24.854508
11  feature_ablation    mse  0.100  159.409729   9.544050


# Others WIP

In [ ]:
batch_x, batch_y, batch_x_mark, batch_y_mark = next(iter(dataloader))
batch_x = batch_x.float().to(exp.device)
batch_y = batch_y.float().to(exp.device)

batch_x_mark = batch_x_mark.float().to(exp.device)
batch_y_mark = batch_y_mark.float().to(exp.device)
# decoder input
dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float()

## FIT

In [35]:
data = get_total_data(dataloader, exp.device, add_x_mark=False)
explainer = Fit(model, features=data)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type                  | Params
-----------------------------------------------
0 | net  | JointFeatureGenerator | 38.6 K
-----------------------------------------------
38.6 K    Trainable params
0         Non-trainable params
38.6 K    Total params
0.154     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=300` reached.


In [36]:
attr = explainer.attribute(
    batch_x, additional_forward_args=(batch_x_mark, dec_inp, batch_y_mark)
)

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [32, 2] but got: [32, 36].

## Dynamask

In [37]:
explainer = DynaMask(model)

In [38]:
attr = explainer.attribute(
    inputs=(batch_x, batch_x_mark),
    additional_forward_args=(dec_inp, batch_y_mark)
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


AssertionError: Multiple inputs are not accepted for this method

## Commented out

In [29]:
# mse_error = mse(
#     model, inputs=batch_x, 
#     attributions=temp, baselines=0, 
#     additional_forward_args=(batch_x_mark, dec_inp, batch_y_mark),
#     topk=0.2
# )
# print(mse_error)

# mae_error = mae(
#     model, inputs=batch_x, 
#     attributions=temp, baselines=0, 
#     additional_forward_args=(batch_x_mark, dec_inp, batch_y_mark),
#     # target=0,
#     topk=0.2
# )
# print(mae_error)

In [30]:
# temporal_mask = torch.zeros_like(batch_x, dtype=int)
# for t in range(batch_x.shape[1]):
#     temporal_mask[:, t] = t

# explainer = FeatureAblation(model)
# time_score = explainer.attribute(
#     inputs=(batch_x),
#     baselines=(batch_x*0),
#     additional_forward_args=(batch_x_mark, dec_inp, batch_y_mark),
#     target=0,
#     feature_mask=temporal_mask
# )
# print(score.shape)